# Spark Kafka Data Streaming

In [ ]:
import os
import pandas as pd
import json
from pyspark.sql.functions import udf, col, from_json, from_csv, sum as _sum
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, DateType

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.spark:spark-avro_2.12:3.3.1 pyspark-shell'


In [ ]:
## Configuration file reader

def read_config(config_file):
    """
    Reads the kafka configuration information that is stored in the system    
    """
    conf = {}    
    with open(config_file) as fh:
        for line in fh:
            line = line.strip()
            if len(line) != 0 and line[0] != "#":
                parameter, value = line.strip().split('=', 1)
                conf[parameter] = value.strip()          
    return conf

In [ ]:
# read the local configuration files

config_path = os.path.join(os.path.dirname('/home/ozkary/.kafka/'),'localhost-nosasl.properties')
config = read_config(config_path)
print(config)

In [ ]:
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark = SparkSession \
    .builder \
    .appName("Spark-Notebook") \
    .getOrCreate()

## Read from Kafka Stream

In [ ]:

topic = 'mta-turnstile'
client_id = 'Spark-Notebook-Session'
group_id = 'turnstile'

use_sasl = "sasl.mechanism" in config and config["sasl.mechanism"] is not None

kafka_options = {
            "kafka.bootstrap.servers": config["bootstrap.servers"],
            "subscribe": topic,
            "startingOffsets": "latest",
            "failOnDataLoss": "false",
            "client.id": client_id,            
            "group.id": group_id,            
            "auto.offset.reset": "latest",
            "checkpointLocation": "checkpoint",
            "minPartitions": "2",
            "enable.auto.commit": "false",
            "enable.partition.eof": "true"                        
        }          

if use_sasl:
    # set the JAAS configuration only when use_sasl is True
    sasl_config = f'org.apache.kafka.common.security.plain.PlainLoginModule required serviceName="kafka" username="{self.settings["sasl.username"]}" password="{self.settings["sasl.password"]}";'

    login_options = {
        "kafka.sasl.mechanisms": self.settings["sasl.mechanism"],
        "kafka.security.protocol": self.settings["security.protocol"],
        "kafka.sasl.username": self.settings["sasl.username"],
        "kafka.sasl.password": self.settings["sasl.password"],  
        "kafka.sasl.jaas.config": sasl_config          
    }
    # merge the login options with the kafka options
    kafka_options = {**kafka_options, **login_options}  

In [ ]:
def value_deserializer(value: bytes) -> any:
    """
    Message value deserializer
    """
    return json.loads(value) 

# set the stream source
# default for startingOffsets is "latest"
stream = spark \
    .readStream \
    .format("kafka") \
    .options(**kafka_options) \
    .option("key.deserializer", value_deserializer) \
    .option("value.deserializer", value_deserializer) \
    .load()


stream.printSchema()

In [ ]:

def write_to_console(df: DataFrame, output_mode: str = 'append', processing_time: str = '15 seconds') -> None:
    """
        Output stream values to the console
    """
    
    console_query = df.writeStream\
        .outputMode(output_mode) \
        .trigger(processingTime=processing_time) \
        .format("console") \
        .option("truncate", False) \
        .start()
    
    # console_query.awaitTermination()   

# write a streaming data frame to storage ./storage
def write_to_storage(df: DataFrame, output_mode: str = 'append', processing_time: str = '15 seconds') -> None:
    """
        Output stream values to the console
    """

    # if "window.start" in df.columns and "window.end" in df.columns:
    #     df_csv = df.select(
    #         col("window.start").alias("START_DT"),
    #         col("window.end").alias("END_DT"),
    #         "A/C", "UNIT", "SCP", "STATION", "LINENAME", "DIVISION", "DATE", "DESC",
    #         "ENTRIES", "EXITS"
    #     )
    # else:
    df_csv = df.select(
        "A/C", "UNIT", "SCP", "STATION", "LINENAME", "DIVISION", "DATE", "DESC",
        "ENTRIES", "EXITS"
    )
        
    # .partitionBy("STATION") \
    storage_query = df_csv.writeStream \
        .outputMode(output_mode) \
        .trigger(processingTime=processing_time) \
        .format("csv") \
        .option("header", True) \
        .option("path", "./storage") \
        .option("checkpointLocation", "./checkpoint") \
        .option("truncate", False) \
        .start()
    
    # storage_query.awaitTermination()

# Define the schema for the incoming data
turnstiles_schema = StructType([
    StructField("`A/C`", StringType()),
    StructField("UNIT", StringType()),
    StructField("SCP", StringType()),
    StructField("STATION", StringType()),
    StructField("LINENAME", StringType()),
    StructField("DIVISION", StringType()),
    StructField("DATE", StringType()),
    StructField("TIME", StringType()),
    StructField("DESC", StringType()),
    StructField("ENTRIES", IntegerType()),
    StructField("EXITS", IntegerType()),
    StructField("ID", StringType()),
    StructField("TIMESTAMP", TimestampType())
])

In [ ]:

def parse_messages(stream, schema) -> DataFrame:
    """
    Parse the messages and use the provided schema to type cast the fields
    """
    assert stream.isStreaming is True, "DataFrame doesn't receive streaming data"

    options =  {'header': 'true', 'sep': ','}
    df = stream.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")               
                                
    # print("df =====>",df)
    # split attributes to nested array in one Column
    col = F.split(df['value'], ',')

    # expand col to multiple top-level columns
    for idx, field in enumerate(schema):
        df = df.withColumn(field.name.replace('`',''), col.getItem(idx).cast(field.dataType))

    result = df.select([field.name for field in schema])    
    result.printSchema()
    
    return result
    

In [ ]:
def agg_messages(df, window_duration: str, window_slide: str) -> DataFrame:
        """
            Window for n minutes aggregations group by A/C, UNIT, STATION, DATE, DESC
        """
        # df = df.na.fill(0)

        # Filter out empty rows
        # df = df.filter(col("A/C").isNotNull() & col("UNIT").isNotNull() & col("STATION").isNotNull())

        # .withWatermark("TIMESTAMP", window_duration) \
        # .groupBy(F.window("TIMESTAMP", window_duration, window_slide),"A/C", "UNIT","SCP","LINENAME","DIVISION", "STATION", "DATE", "DESC") \

        df_windowed = df \
            .groupBy(F.window("TIMESTAMP", window_duration, window_slide),"A/C", "UNIT","SCP","LINENAME","DIVISION", "STATION", "DATE", "DESC") \
            .agg(
                _sum("ENTRIES").alias("ENTRIES"),
                _sum("EXITS").alias("EXITS")
            )    
        
        # df_windowed.printSchema()    
        print("df_windowed =====>",df_windowed)        
        # df_windowed.show(10, False)
        

        return df_windowed

In [ ]:
def add_by_station(df, window_duration: str, window_slide: str) -> DataFrame:
    
      # Ensure TIMESTAMP is in the correct format (timestamp type)
    df = df.withColumn("TIMESTAMP", F.col("TIMESTAMP").cast("timestamp"))
    
    # Group by 'STATION' and create a 2-minute window
    df_windowed = df.groupBy(df["STATION"], F.window(df["TIMESTAMP"], window_duration)).agg(
        F.sum(df["ENTRIES"]).alias("ENTRIES"),
        F.sum(df["EXITS"]).alias("EXITS")
    ).withColumn("START", F.col("window.start")).withColumn("END", F.col("window.end"))

    # Drop the original window column
    df_windowed.drop("window")

    df_windowed.printSchema()
    return df_windowed 

In [ ]:
# convert the schema to string
schema_string = turnstiles_schema.simpleString()
df_messages = parse_messages(stream, schema=turnstiles_schema)
write_to_console(df_messages)

# write_to_storage(df_messages)

window_duration = '2 minutes'
window_slide = '2 minutes'

df_windowed = agg_messages(df_messages,window_duration, window_slide)
    
# add a type column to both dataframes to be able to join them
# df_messages_with_type = df_messages.withColumn("type", F.lit("turnstile"))
# df_windowed_with_type = df_windowed.withColumn("type", F.lit("windowed"))

# union the dataframes to produce a single console output
# df_union = df_messages_with_type.union(df_windowed_with_type)

# write_to_console(df_messages_with_type)

write_to_console(df_windowed)

# write_to_storage(df_windowed)

# wait 3 minutes and then stop the query
# time.sleep(180)
# console_query.stop()
# storage_query.stop()

# read all the csv files from storage and show the data
# df = spark.read.csv('./storage/*.csv', header=True)
# df.show()


-------------------------------------------
Batch: 341
-------------------------------------------
-------------------------------------------
Batch: 322
-------------------------------------------
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+
|A/C  |UNIT|SCP     |STATION     |LINENAME|DIVISION|DATE    |TIME    |DESC   |ENTRIES|EXITS|ID                                  |TIMESTAMP|
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+
|"A001|R002|02-00-00|Test-Station|456NQR  |BMT     |11-28-23|15:03:41|REGULAR|808    |643  |beac6390-1589-46e6-aa54-a8a65d52b533|null     |
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+

-------------------------------------------
Batch: 240
-------------------------------------------
+-

-------------------------------------------
Batch: 340
-------------------------------------------
-------------------------------------------
Batch: 242
-------------------------------------------
-------------------------------------------
Batch: 324
-------------------------------------------
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+
|A/C  |UNIT|SCP     |STATION     |LINENAME|DIVISION|DATE    |TIME    |DESC   |ENTRIES|EXITS|ID                                  |TIMESTAMP|
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+
|"A001|R002|02-00-00|Test-Station|456NQR  |BMT     |11-28-23|15:04:01|REGULAR|562    |901  |16a00421-f71a-46dd-be37-1f3ede0f470a|null     |
|"A001|R001|02-00-00|Test-Station|456NQR  |BMT     |11-28-23|15:04:11|REGULAR|526    |662  |896ec03a-9e82-48d7-bc65-8eb6eadf8cd1|null     |
+--

-------------------------------------------
Batch: 349
-------------------------------------------
-------------------------------------------
Batch: 251
-------------------------------------------
-------------------------------------------
Batch: 333
-------------------------------------------
-------------------------------------------
Batch: 323
-------------------------------------------
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+
|A/C  |UNIT|SCP     |STATION     |LINENAME|DIVISION|DATE    |TIME    |DESC   |ENTRIES|EXITS|ID                                  |TIMESTAMP|
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+
|"A001|R001|02-00-00|Test-Station|456NQR  |BMT     |11-28-23|15:06:21|REGULAR|543    |511  |fb4ceb85-5e40-4624-ba5b-6cd569e3a880|null     |
+-----+----+--------+------------+--------+-

-------------------------------------------
Batch: 335
-------------------------------------------
-------------------------------------------
Batch: 351
-------------------------------------------
-------------------------------------------
Batch: 354
-------------------------------------------
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+
|A/C  |UNIT|SCP     |STATION     |LINENAME|DIVISION|DATE    |TIME    |DESC   |ENTRIES|EXITS|ID                                  |TIMESTAMP|
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+
|"A001|R002|02-00-00|Test-Station|456NQR  |BMT     |11-28-23|15:06:51|REGULAR|642    |813  |2b362b15-92f6-4730-910f-50c796641cbf|null     |
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+

+-

-------------------------------------------
Batch: 334
-------------------------------------------
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+
|A/C  |UNIT|SCP     |STATION     |LINENAME|DIVISION|DATE    |TIME    |DESC   |ENTRIES|EXITS|ID                                  |TIMESTAMP|
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+
|"A001|R001|02-00-00|Test-Station|456NQR  |BMT     |11-28-23|15:09:01|REGULAR|715    |831  |d96b02b1-f030-40a6-8c6a-8b1a8f63ea1d|null     |
|"A002|R002|02-00-00|Test-Station|456NQR  |BMT     |11-28-23|15:09:11|REGULAR|539    |866  |3c8c4665-6890-4c25-83ce-daa568b123e3|null     |
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+

-------------------------------------------
Batch: 360
-----

...

-------------------------------------------
Batch: 509
-------------------------------------------
-------------------------------------------
Batch: 411
-------------------------------------------
-------------------------------------------
Batch: 493
-------------------------------------------
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+
|A/C  |UNIT|SCP     |STATION     |LINENAME|DIVISION|DATE    |TIME    |DESC   |ENTRIES|EXITS|ID                                  |TIMESTAMP|
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+
|"A002|R002|02-00-00|Test-Station|456NQR  |BMT     |11-28-23|15:46:23|REGULAR|570    |882  |195be1e9-56a5-49a3-a0a8-93a0cd866a09|null     |
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+

--

-------------------------------------------
Batch: 485
-------------------------------------------
-------------------------------------------
Batch: 495
-------------------------------------------
-------------------------------------------
Batch: 514
-------------------------------------------
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+
|A/C  |UNIT|SCP     |STATION     |LINENAME|DIVISION|DATE    |TIME    |DESC   |ENTRIES|EXITS|ID                                  |TIMESTAMP|
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+
|"A002|R002|02-00-00|Test-Station|456NQR  |BMT     |11-28-23|15:46:53|REGULAR|917    |727  |898c2dd1-fcf5-47b6-a632-ece6f0a386a5|null     |
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+

+-

-------------------------------------------
Batch: 414
-------------------------------------------
-------------------------------------------
Batch: 512
-------------------------------------------
-------------------------------------------
Batch: 486
-------------------------------------------
-------------------------------------------
Batch: 515
-------------------------------------------
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+
|A/C  |UNIT|SCP     |STATION     |LINENAME|DIVISION|DATE    |TIME    |DESC   |ENTRIES|EXITS|ID                                  |TIMESTAMP|
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+
|"A002|R002|02-00-00|Test-Station|456NQR  |BMT     |11-28-23|15:47:03|REGULAR|989    |605  |4b718b73-2171-4956-bacc-fef95b17d552|null     |
|"A002|R002|02-00-00|Test-Station|456NQR  |B

-------------------------------------------
Batch: 488
-------------------------------------------
-------------------------------------------
Batch: 517
-------------------------------------------
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+
|A/C  |UNIT|SCP     |STATION     |LINENAME|DIVISION|DATE    |TIME    |DESC   |ENTRIES|EXITS|ID                                  |TIMESTAMP|
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+
|"A001|R001|02-00-00|Test-Station|456NQR  |BMT     |11-28-23|15:55:40|REGULAR|873    |681  |fb6d38e8-1cfb-460d-a665-d963a1cde805|null     |
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+

+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+---------

-------------------------------------------
Batch: 521
-------------------------------------------
-------------------------------------------
Batch: 492
-------------------------------------------
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+
|A/C  |UNIT|SCP     |STATION     |LINENAME|DIVISION|DATE    |TIME    |DESC   |ENTRIES|EXITS|ID                                  |TIMESTAMP|
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+------------------------------------+---------+
|"A002|R002|02-00-00|Test-Station|456NQR  |BMT     |11-28-23|15:56:30|REGULAR|946    |590  |8d8f9349-3e3b-4f93-88ed-23b152327005|null     |
|"A001|R001|02-00-00|Test-Station|456NQR  |BMT     |11-28-23|15:56:40|REGULAR|895    |839  |1a0fcdf8-5f95-41c0-8aee-3d46eedccc81|null     |
+-----+----+--------+------------+--------+--------+--------+--------+-------+-------+-----+----------